In [1]:
import pandas as pd
import numpy as np

from pycaret.classification import *

from sklearn.metrics import log_loss

In [2]:
data_raw = catalog.load("raw_data_kobe")
data_raw

[04/05/23 16:30:44] INFO     Loading data from 'raw_data_kobe' (CSVDataSet)...                  ]8;id=196773;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=745450;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

,action_type,combined_shot_type,game_event_id,game_id,lat,loc_x,loc_y,lon,minutes_remaining,period,...,shot_type,shot_zone_area,shot_zone_basic,shot_zone_range,team_id,team_name,game_date,matchup,opponent,shot_id
0,Jump Shot,Jump Shot,10,20000012,33.9723,167,72,-118.1028,10,1,...,2PT Field Goal,Right Side(R),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,1
1,Jump Shot,Jump Shot,12,20000012,34.0443,-157,0,-118.4268,10,1,...,2PT Field Goal,Left Side(L),Mid-Range,8-16 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,2
2,Jump Shot,Jump Shot,35,20000012,33.9093,-101,135,-118.3708,7,1,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,3
3,Jump Shot,Jump Shot,43,20000012,33.8693,138,175,-118.1318,6,1,...,2PT Field Goal,Right Side Center(RC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,4
4,Driving Dunk Shot,Dunk,155,20000012,34.0443,0,0,-118.2698,6,2,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-10-31,LAL @ POR,POR,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30692,Jump Shot,Jump Shot,397,49900088,33.9963,1,48,-118.2688,6,4,...,2PT Field Goal,Center(C),In The Paint (Non-RA),Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30693
30693,Tip Shot,Tip Shot,398,49900088,34.0443,0,0,-118.2698,6,4,...,2PT Field Goal,Center(C),Restricted Area,Less Than 8 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30694
30694,Running Jump Shot,Jump Shot,426,49900088,33.8783,-134,166,-118.4038,3,4,...,2PT Field Goal,Left Side Center(LC),Mid-Range,16-24 ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30695
30695,Jump Shot,Jump Shot,448,49900088,33.7773,31,267,-118.2388,2,4,...,3PT Field Goal,Center(C),Above the Break 3,24+ ft.,1610612747,Los Angeles Lakers,2000-06-19,LAL vs. IND,IND,30696


In [3]:
data_intermediate = catalog.load("data_filtered")
data_intermediate

[04/05/23 16:30:47] INFO     Loading data from 'data_filtered' (ParquetDataSet)...              ]8;id=294276;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=811668;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
0,33.9723,-118.1028,10,1,0,18,NaN
1,34.0443,-118.4268,10,1,0,15,0.0
2,33.9093,-118.3708,7,1,0,16,1.0
3,33.8693,-118.1318,6,1,0,22,0.0
4,34.0443,-118.2698,6,2,0,0,1.0
...,...,...,...,...,...,...,...
30691,34.0443,-118.2698,7,4,1,0,0.0
30692,33.9963,-118.2688,6,4,1,4,0.0
30693,34.0443,-118.2698,6,4,1,0,NaN
30694,33.8783,-118.4038,3,4,1,21,1.0


In [4]:
data_primary = catalog.load("data_conformed")
data_primary

[04/05/23 16:30:51] INFO     Loading data from 'data_conformed' (ParquetDataSet)...             ]8;id=679705;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=592116;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
1,34.0443,-118.4268,10,1,0,15,0.0
2,33.9093,-118.3708,7,1,0,16,1.0
3,33.8693,-118.1318,6,1,0,22,0.0
4,34.0443,-118.2698,6,2,0,0,1.0
5,34.0553,-118.4148,9,3,0,14,0.0
...,...,...,...,...,...,...,...
30690,33.9443,-118.3828,11,4,1,15,0.0
30691,34.0443,-118.2698,7,4,1,0,0.0
30692,33.9963,-118.2688,6,4,1,4,0.0
30694,33.8783,-118.4038,3,4,1,21,1.0


In [5]:
data_primary['shot_made_flag'].value_counts()

0.0    10602
1.0     9683
Name: shot_made_flag, dtype: int64

In [6]:
# Base está balanceada
data_primary['shot_made_flag'].value_counts(normalize=True)

0.0    0.522652
1.0    0.477348
Name: shot_made_flag, dtype: float64

In [7]:
base_train = catalog.load("base_train")
base_train

[04/05/23 16:31:01] INFO     Loading data from 'base_train' (ParquetDataSet)...                 ]8;id=228736;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=57613;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
11476,34.0443,-118.2698,2,2,0,0,0.0
26552,34.0373,-118.3888,7,3,1,11,0.0
1209,34.0443,-118.2698,2,3,0,0,1.0
28269,34.0133,-118.4028,5,4,1,13,0.0
10504,34.0453,-118.1038,2,1,0,16,1.0
...,...,...,...,...,...,...,...
12390,34.0443,-118.2698,11,2,0,0,0.0
15071,33.9773,-118.4788,5,2,0,21,0.0
27609,33.9553,-118.2598,0,3,1,8,1.0
13,34.0403,-118.3638,2,1,0,9,0.0


In [8]:
base_test = catalog.load("base_test")
base_test

[04/05/23 16:31:03] INFO     Loading data from 'base_test' (ParquetDataSet)...                  ]8;id=666233;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=231462;file://D:\.venv\venv_black_mamba_pd\lib\site-packages\kedro\io\data_catalog.py#343\343]8;;\

,lat,lon,minutes_remaining,period,playoffs,shot_distance,shot_made_flag
6494,34.0443,-118.2698,5,4,0,0,1.0
2113,34.0313,-118.3028,7,1,0,3,0.0
5061,34.0493,-118.1798,2,2,0,9,0.0
24559,33.9873,-118.3828,7,1,0,12,0.0
30414,34.0443,-118.2698,3,3,1,0,0.0
...,...,...,...,...,...,...,...
21484,33.9073,-118.1448,4,1,0,18,1.0
24648,34.0653,-118.3788,7,4,0,11,1.0
1260,34.0443,-118.2698,2,3,0,0,0.0
10221,33.9983,-118.2488,5,2,0,5,0.0
